# <center>Final Report on the Project  
# <center>[Safe Trade: Stock Prediction](https://github.com/breindy/SafeTrade)
Shofiqur Rahman<br>
Bashir Ahamed<br>
Michelle Uy<br>
Thierno Diallo<br><br>
Grant Long<br>
CSC.59970<br>
December 10, 2018

For the purpose of creating the model, we had to spend a lot of time on just data gathering and cleaning. Since these finance data could not be found anywhere ready as to be used in modelling, we had to query into different API's to get the data and have them ready for analysis. Most of the data came from [IEX API](https://iextrading.com/developer/docs/), but the stock exchange index data came from [Yahoo Finance](https://finance.yahoo.com/quote/%5EIXIC/history?ltr=1). 
We wrote a separate [python script](AggregateData.py) to fetch the necessary data. We had the data cleaned and ready to be used in model building process by the same script. The script basically makes request to the IEX API to fetch all the company symbols. For each symbol that are presently in transaction in the IEX Exchange, the script makes subsequent calls to fetch the whole year eps, dividend, and all other features value and add them to the main dataframe in iterations. 

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv('AllStockData.csv')

#print(df.describe())
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537295 entries, 0 to 537294
Data columns (total 12 columns):
Unnamed: 0         537295 non-null int64
date               537295 non-null object
company            537295 non-null object
symbol             537295 non-null object
dividend           537295 non-null float64
eps                537295 non-null float64
grossProfit        537295 non-null float64
nasdaqIndex        537295 non-null float64
marketcap          537295 non-null int64
pe                 537295 non-null float64
price              537295 non-null float64
revenuePerShare    537295 non-null int64
dtypes: float64(6), int64(3), object(3)
memory usage: 49.2+ MB
None


,Unnamed: 0,date,company,symbol,dividend,eps,grossProfit,nasdaqIndex,marketcap,pe,price,revenuePerShare
0,0,2017-12-04,Zoetis Inc. Class A,ZTS,0.105,0.76,3.068000e+09,6775.370117,45099924293,70.5991,70.5991,3
1,1,2017-12-05,Zoetis Inc. Class A,ZTS,0.105,0.76,3.068000e+09,6762.209961,45099924293,70.4302,70.4302,3
2,2,2017-12-06,Zoetis Inc. Class A,ZTS,0.105,0.76,3.068000e+09,6776.379883,45099924293,70.5494,70.5494,3
3,3,2017-12-07,Zoetis Inc. Class A,ZTS,0.105,0.76,3.068000e+09,6812.839844,45099924293,71.2155,71.2155,3
4,4,2017-12-08,Zoetis Inc. Class A,ZTS,0.105,0.76,3.068000e+09,6840.080078,45099924293,71.4541,71.4541,3


In [20]:
df.replace([np.inf, -np.inf], 100)

df.isnull().values.any()

False

In [21]:
### Cleaning the dataframe df to have only features that have numeric values as well as 
# dividing the DataFrame in X(Features) and Y(Target)
X = df[['dividend', 'eps', 'grossProfit', 'marketcap', 'nasdaqIndex', 'pe', 'revenuePerShare']].copy()
Y = df.price
print(X.head(3))
print(Y.head(3))

   dividend   eps   grossProfit    marketcap  nasdaqIndex       pe  \
0     0.105  0.76  3.068000e+09  45099924293  6775.370117  70.5991   
1     0.105  0.76  3.068000e+09  45099924293  6762.209961  70.4302   
2     0.105  0.76  3.068000e+09  45099924293  6776.379883  70.5494   

   revenuePerShare  
0                3  
1                3  
2                3  
0    70.5991
1    70.4302
2    70.5494
Name: price, dtype: float64


In [22]:
### To see what features contribute the most toward price determination

# Splitting the dataset into train and test so that we have a holdout set to test our model's performance
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

# Instantiate the datamodel rf
rf = RandomForestRegressor(n_estimators=25, random_state=2)

# Fit rf to the training set    
rf.fit(X_train, y_train)

# Predict the test set labels
y_pred = rf.predict(X_test)

# Evaluate the test set RMSE
rmse_test = MSE(y_test, y_pred)**(1/2)

# Print rmse_test
print('Test set RMSE of rf: {:.2f}'.format(rmse_test))

# Create a pd.Series of features importances
importances = pd.Series(data=rf.feature_importances_, index= X_train.columns)

# Sort importances
importances_sorted = importances.sort_values()

# Draw a horizontal barplot of importances_sorted
importances_sorted.plot(kind='barh', color='lightgreen')
plt.title('Features Importances')
plt.show()

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

# Challenges
First of all, to come up with valid features was a challenge. What features we feed into the model to train on. To ease the process of investing, we faced with a challenge of learning finance. Having a finance major in our team could have been very fruitful since we are not so educated with all those finance terms. 
Second, the data was not ready anywhere. Though, we got most of our data from [IEX API](https://iextrading.com/developer/docs/), it did not come in a form to be used in the data model. We had to check for missing values, values not in proper format etc. 


In [ ]:
# Things to consider: stocks will tend to be correlated with each other